In [1]:
import csv
import pandas as pd
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import helpers as analysis
from numpy import linalg as LA
import numpy as np
import re


[nltk_data] Downloading package punkt to /Users/Home/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
df = pd.read_csv('data/data_cleaning_coffee.csv')
df['est_price']

0       $48.71/340 grams
1       $30.78/340 grams
2       $16.00/340 grams
3       $19.00/340 grams
4       $16.50/340 grams
              ...       
2272    $16.00/340 grams
2273    $10.00/340 grams
2274    $10.00/340 grams
2275    $16.00/340 grams
2276    $20.00/340 grams
Name: est_price, Length: 2277, dtype: object

In [3]:
combined_descriptions = df[["desc_1"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_names = df[["name"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_locs = df[["origin"]].apply(lambda x : ' '.join(x.dropna()), axis=1)
combined_prices = df[["est_price"]].apply(lambda x : ' '.join(x.dropna()), axis=1)

combined_descriptions = [x for x in combined_descriptions]
combined_names = [x + " from " + combined_locs[i] for i, x in enumerate(combined_names)]

vectorizer = TfidfVectorizer()


#Replace query with the user query here 
query = ["Nutty and Sweet"]
# query = query.lower()
# query = query.split(" ")

doc_vectors  = vectorizer.fit_transform(query + combined_descriptions).toarray()

index_to_vocab = {i:v for i, v in enumerate(vectorizer.get_feature_names_out())}
doc_to_index = {v:i for i, v in enumerate(combined_names)}
index_to_doc_descriptions = {i:{'name': v, 'description': combined_descriptions[i]}for i, v in enumerate(combined_names)}

cosineSims = (np.dot(doc_vectors[0], np.transpose(doc_vectors[1:]))/(LA.norm(doc_vectors[0])* LA.norm(doc_vectors[1:])))

cosineSims = [(x, i) for i, x in enumerate(cosineSims)]

cosineSims = sorted(cosineSims, key = lambda x:x[0], reverse = True)

topTen = cosineSims[:10]
answer  = []
for sim, index in topTen: 
  answer.append((index_to_doc_descriptions[index], sim))




In [6]:
price = [x for x in combined_prices]
for i, x in enumerate(price):
  price = [float(x[re.find([$]+1:x.find('/'))]) for x in price]
print(price)

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (1687839994.py, line 3)

In [ ]:
search_terms = "my query"
documents = ["my","list","of","docs"]

vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform([search_terms] + documents).toarray()

print(np.dot(doc_vectors[0], np.transpose(doc_vectors[1:]))/(LA.norm(doc_vectors[0])* LA.norm(doc_vectors[1:])))

# document_scores = [item.item() for item in cosine_similarities[1:]]

[0.31395688 0.         0.         0.        ]


In [ ]:
def findTopTen(user_query):
    """
    Takes a user query and returns an array of (dictionary, integer) pairs
    Dictionary is a dictionary of 'name', and 'description' of a coffee, and the integer is the cosine similarity

    Result is returned in order of cosine similarity highest to lowest
    User query must be input as a string
    """

    import csv
    import pandas as pd
    import nltk
    from nltk.tokenize import word_tokenize
    from nltk.stem import PorterStemmer
    from collections import Counter
    import string
    from sklearn.feature_extraction.text import TfidfVectorizer
    import helpers as analysis
    from numpy import linalg as LA
    import numpy as np

    df = pd.read_csv("data/data_cleaning_coffee.csv")

    combined_descriptions = df[["desc_1"]].apply(lambda x: " ".join(x.dropna()), axis=1)
    combined_names = df[["name"]].apply(lambda x: " ".join(x.dropna()), axis=1)
    combined_locs = df[["origin"]].apply(lambda x: " ".join(x.dropna()), axis=1)

    combined_descriptions = [x for x in combined_descriptions]
    combined_names = [
        x + " from " + combined_locs[i] for i, x in enumerate(combined_names)
    ]

    vectorizer = TfidfVectorizer()

    # Replace query with the user query here
    query = [user_query]

    doc_vectors = vectorizer.fit_transform(query + combined_descriptions).toarray()
    # don't think we need these, but in the event that we have a very slow query, we can use this
    # index_to_vocab = {i: v for i, v in enumerate(vectorizer.get_feature_names_out())}
    # doc_to_index = {v: i for i, v in enumerate(combined_names)}
    index_to_doc_descriptions = {
        i: {"name": v, "description": combined_descriptions[i]}
        for i, v in enumerate(combined_names)
    }

    cosineSims = np.dot(doc_vectors[0], np.transpose(doc_vectors[1:])) / (
        LA.norm(doc_vectors[0]) * LA.norm(doc_vectors[1:])
    )

    cosineSims = [(x, i) for i, x in enumerate(cosineSims)]

    cosineSims = sorted(cosineSims, key=lambda x: x[0], reverse=True)

    topTen = cosineSims[:10]
    answer = []
    for sim, index in topTen:
        answer.append((index_to_doc_descriptions[index], sim))

    return answer


In [ ]:
answer = findTopTen("sweet and floral")

for i, x in enumerate(answer): 
  print(x[0]['name'])
  print(x[0]['description'])
  print(x[1])


Finca Lerida Gesha Natural from Jaramillo, Boquete growing region, western Panama
Richly sweet, floral-toned. Magnolia, date, almond, caramel, fresh-cut cedar in aroma and cup. Sweet structure with gentle, round acidity; full, syrupy-smooth mouthfeel. Floral and nut-toned finish.
0.009647394581102009
Costa Rica Central Valley Brandy Ball from Central Valley, Costa Rica
Richly sweet, floral-toned. Freesia-like flowers, orange zest, hazelnut, caramel, cedar in aroma and cup. Sweet structure with gentle, round acidity; full, syrupy-smooth mouthfeel. Crisp floral- and nut-toned finish.
0.009341387794646074
Panama Geisha from Jaramillo, Boquete growing region, Panama
Richly sweet, floral-toned. Honeysuckle, raisin, pistachio, caramel, fresh-cut cedar in aroma and cup. Sweet-toned structure with gentle, round acidity; velvety-smooth mouthfeel. Floral and nut-toned finish.
0.008860441548207946
Honduras Relationship Coffee Norma Lara from Ocotepeque Department, Honduras
Richly floral-toned, de